# Clustering

![so easy for us](assets/clustering/robot-having-trouble.jpg)

(image: [toptal](https://www.toptal.com/machine-learning/clustering-algorithms))

## Where are we?

![yay, one more quadrant covered](assets/linear-regression/machine-learning-cheet-sheet.png)

(image: [sas.com](https://www.sas.com/en_us/insights/analytics/machine-learning.html))

## Supervised / Unsupervised Learning

Unsupervised learning:
- Just dataset, no targets or labels
- Algorithm needs to "make sense" of the data itself
- E.g. clustering, dimension reduction

Supervised learning:
- Dataset with targets or labels
- Algorithm learns by minimizing loss against the targets / labels
- E.g. Classification, regression

![supervised vs unsupervised](assets/clustering/supervised_unsupervised.png)

(image: [Vibhor Agarwal](https://medium.com/@agarwalvibhor84/getting-started-with-machine-learning-using-sklearn-python-7d165618eddf))

## Clustering

Objective: given a dataset with just features, find groups (or clusters) of them

Broad applications:
- Market segmentation
- User recommendations
- Anormaly detection

https://en.wikipedia.org/wiki/Cluster_analysis#Applications

## Algorithms

- K-means / K-modes
- Expectation Maximization / Gaussian Mixture
- DBSCAN

Comparison:
http://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods

## K-means

## Gaussian Mixture

## DBSCAN

## Evaluation Metrics

http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation
